In [ ]:
# student_proj, student_out = student(images)
# student_proj.shape [Batch_size, feature_size]
# student_out.shape [Batch_size, class_num]
# f_i表示student_proj[i]，p_i表示student_out[i]，p_i^k表示student_out[i][k]，代表第i个对象属于第k个类别的预测值
# \overline{f}_{k}=\sum_{i} \frac{p_{i}^{k}}{\sum_{k}p_{i}}\cdot f_{i}

In [8]:
import numpy as np

# 假设有以下的小批量数据
# student_proj: 3个对象，每个对象的特征大小为2
student_proj = np.array([[0.2, 0.7],
                         [0.5, 0.1],
                         [0.9, 0.8],
                         [0.1, 0.8]])

# student_out: 3个对象，每个对象有3个类别的预测值
student_out = np.array([[0.1, 0.3, 0.6],
                        [0.5, 0.2, 0.3],
                        [0.2, 0.4, 0.4],
                        [0.2, 0.4, 0.4]])

# 计算归一化的预测权重
normalized_student_out = student_out / student_out.sum(axis=1, keepdims=True)

# 计算加权的特征值
weighted_features = normalized_student_out.T @ student_proj

print("Normalized Student Outputs:")
print(normalized_student_out)
print("\nWeighted Features for each class:")
print(weighted_features)


Normalized Student Outputs:
[[0.1 0.3 0.6]
 [0.5 0.2 0.3]
 [0.2 0.4 0.4]
 [0.2 0.4 0.4]]

Weighted Features for each class:
[[0.47 0.44]
 [0.56 0.87]
 [0.67 1.09]]


In [ ]:
# student_proj, student_out = student(images)
# student_proj.shape [Batch_size, feature_size]
# student_out.shape [Batch_size, class_num]
# 对于student_out，我希望得到每一行最小的k个值的对应的类别（索引）

In [11]:
# 示例数据
# student_out = np.array([[0.1, 0.3, 0.6, 0.8, 0.05],
#                         [0.5, 0.2, 0.3, 0.7, 0.1],
#                         [0.2, 0.4, 0.4, 0.1, 0.6]])

k = 2  # 作为示例，选择最小的2个值

# 使用 argsort 获取索引，并选择前 k 个最小值的索引
smallest_k_indices = np.argsort(student_out)[:, :k]

print("Indices of the smallest k values for each row:")
print(smallest_k_indices)


Indices of the smallest k values for each row:
[[0 1]
 [1 2]
 [0 1]
 [0 1]]


In [12]:
# student_proj.shape [Batch_size, feature_size], weighted_features.shape [class_num, feature_size], smallest_k_indices[Batch_size, k]
# 基于这些变量实现 \sum_{k\in U}\frac{1}{||f_{i}-\overline{f}_{k}||_{2}^{2}}
# 其中f_{i}=student_proj[i], \overline{f}_{k}=weighted_features[k], U=smallest_k_indices[i]
results = []

for i, f_i in enumerate(student_proj):
    U = smallest_k_indices[i]
    distances = [np.linalg.norm(f_i - weighted_features[k])**2 for k in U]
    result_for_i = np.sum([1/d for d in distances if d > 1e-8])  # 避免除以0
    results.append(result_for_i)

results = np.array(results)  # Shape: [Batch_size]

print(results)


[13.42658599  2.66752621 11.47840543  8.37128286]


In [13]:
# 包装一下
import numpy as np

def compute_results(student_proj, student_out, k):
    # 计算归一化的预测权重
    normalized_student_out = student_out / student_out.sum(axis=1, keepdims=True)

    # 计算加权的特征值
    weighted_features = normalized_student_out.T @ student_proj

    # 使用 argsort 获取索引，并选择前 k 个最小值的索引
    smallest_k_indices = np.argsort(student_out)[:, :k]

    results = []
    for i, f_i in enumerate(student_proj):
        U = smallest_k_indices[i]
        distances = [np.linalg.norm(f_i - weighted_features[k])**2 for k in U]
        result_for_i = np.sum([1/d for d in distances if d > 1e-8])  # 避免除以0
        results.append(result_for_i)

    return np.array(results)  # Shape: [Batch_size]

# student_proj: 3个对象，每个对象的特征大小为2
student_proj = np.array([[0.2, 0.7],
                         [0.5, 0.1],
                         [0.9, 0.8],
                         [0.1, 0.8]])

# student_out: 3个对象，每个对象有3个类别的预测值
student_out = np.array([[0.1, 0.3, 0.6],
                        [0.5, 0.2, 0.3],
                        [0.2, 0.4, 0.4],
                        [0.2, 0.4, 0.4]])

k = 2
results = compute_results(student_proj, student_out, k)
print(results)


[13.42658599  2.66752621 11.47840543  8.37128286]


In [1]:
import torch

def compute_results(student_proj, student_out, k):
    # 计算归一化的预测权重
    normalized_student_out = student_out / student_out.sum(dim=1, keepdim=True)

    # 计算加权的特征值
    weighted_features = torch.matmul(normalized_student_out.t(), student_proj)

    # 使用 argsort 获取索引，并选择前 k 个最小值的索引
    smallest_k_indices = student_out.argsort(dim=1)[:, :k]

    results = []
    for i, f_i in enumerate(student_proj):
        U = smallest_k_indices[i]
        distances = [torch.norm(f_i - weighted_features[k]).pow(2) for k in U]
        result_for_i = torch.sum(torch.tensor([1/d for d in distances if d > 1e-8]))  # 避免除以0
        results.append(result_for_i)

    return torch.stack(results)  # Shape: [Batch_size]

# 使用示例
student_proj = torch.tensor([[0.2, 0.7],
                         [0.5, 0.1],
                         [0.9, 0.8],
                         [0.1, 0.8]])
student_out = torch.tensor([[0.1, 0.3, 0.6],
                        [0.5, 0.2, 0.3],
                        [0.2, 0.4, 0.4],
                        [0.2, 0.4, 0.4]])

k = 2
results = compute_results(student_proj, student_out, k)
print(results)


/opt/conda/envs/gcd/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([13.4266,  2.6675, 11.4784,  8.3713])


In [4]:
import torch

def compute_results(student_proj, student_out, k):
    """
    Compute results based on the student projections and outputs.
    
    :param student_proj: Tensor of student projections. Shape: [batch_size, feature_size]
    :param student_out: Tensor of student outputs. Shape: [batch_size, num_classes]
    :param k: Number of smallest values to consider.
    
    :return: Tensor of results. Shape: [batch_size]
    """

    # Step 1: Normalize student outputs
    normalized_student_out = student_out / student_out.sum(dim=1, keepdim=True)

    # Step 2: Calculate weighted features
    weighted_features = torch.matmul(normalized_student_out.t(), student_proj)

    # Step 3: Get smallest k indices from student outputs
    smallest_k_indices = student_out.argsort(dim=1)[:, :k]

    # Step 4: Compute distances
    expanded_proj = student_proj.unsqueeze(1).expand(-1, k, -1)
    indexed_weighted_features = torch.gather(weighted_features.unsqueeze(0).expand(student_proj.size(0), -1, -1), 
                                             1, smallest_k_indices.unsqueeze(2).expand(-1, -1, student_proj.size(1)))
    distances = torch.norm(expanded_proj - indexed_weighted_features, dim=2).pow(2)

    # Step 5: Calculate final results
    results = torch.sum(torch.where(distances > 1e-8, 1.0 / distances, torch.zeros_like(distances)), dim=1)

    return results

# 使用示例
student_proj = torch.tensor([[0.2, 0.7],
                             [0.5, 0.1],
                             [0.9, 0.8],
                             [0.1, 0.8]])
student_out = torch.tensor([[0.1, 0.3, 0.6],
                            [0.5, 0.2, 0.3],
                            [0.2, 0.4, 0.4],
                            [0.2, 0.4, 0.4]])

k = 2
results = compute_results(student_proj, student_out, k)
print(results)


tensor([13.4266,  2.6675, 11.4784,  8.3713])


In [1]:
# 载入SimGCD模型
import torch
import sys
sys.path.append("/wang_hp/zhy/SimGCD-main")
from model import DINOHead
import torch.nn as nn

backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
projector = DINOHead(in_dim=768, out_dim=200, nlayers=3)
model = nn.Sequential(backbone, projector).to('cuda')
checkpoint = torch.load('/wang_hp/zhy/SimGCD-main/dev_outputs/simgcd/log/init_test_(04.09.2023_|_50.012)/checkpoints/model.pt')
model.load_state_dict(checkpoint['model'])

/opt/conda/envs/gcd/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


<All keys matched successfully>

In [2]:
# 载入验证集
from torchvision import transforms
from PIL import ImageOps, ImageFilter
import random
import pandas as pd
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset
from copy import deepcopy
import numpy as np
import os
from torch.utils.data import DataLoader

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
image_size = 224
interpolation = 3
crop_pct = 0.875
test_transform = transforms.Compose([
    transforms.Resize(int(image_size / crop_pct), interpolation),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=torch.tensor(mean),
        std=torch.tensor(std))
])

class CustomCub2011(Dataset):
    base_folder = 'CUB_200_2011/images'
    url = 'http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz'
    filename = 'CUB_200_2011.tgz'
    tgz_md5 = '97eceeb196236b17998738112f37df78'

    def __init__(self, root='/wang_hp/zhy/gcd-task/data', train=True, transform=None, target_transform=None, loader=default_loader, download=True):

        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform

        self.loader = loader
        self.train = train


        if download:
            self._download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        self.uq_idxs = np.array(range(len(self)))

    def _load_metadata(self):
        images = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'images.txt'), sep=' ',
                             names=['img_id', 'filepath'])
        image_class_labels = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'image_class_labels.txt'),
                                         sep=' ', names=['img_id', 'target'])
        train_test_split = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'train_test_split.txt'),
                                       sep=' ', names=['img_id', 'is_training_img'])

        data = images.merge(image_class_labels, on='img_id')
        self.data = data.merge(train_test_split, on='img_id')

        if self.train:
            self.data = self.data[self.data.is_training_img == 1]
        else:
            self.data = self.data[self.data.is_training_img == 0]

    def _check_integrity(self):
        try:
            self._load_metadata()
        except Exception:
            return False

        for index, row in self.data.iterrows():
            filepath = os.path.join(self.root, self.base_folder, row.filepath)
            if not os.path.isfile(filepath):
                print(filepath)
                return False
        return True

    def _download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        download_url(self.url, self.root, self.filename, self.tgz_md5)

        with tarfile.open(os.path.join(self.root, self.filename), "r:gz") as tar:
            tar.extractall(path=self.root)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        path = os.path.join(self.root, self.base_folder, sample.filepath)
        target = sample.target - 1  # Targets start at 1 by default, so shift to 0
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        # return img, target, self.uq_idxs[idx]
        return img, target, idx

def subsample_classes(dataset, include_classes=range(160)):

    include_classes_cub = np.array(include_classes) + 1     # CUB classes are indexed 1 --> 200 instead of 0 --> 199
    cls_idxs = [x for x, (_, r) in enumerate(dataset.data.iterrows()) if int(r['target']) in include_classes_cub]

    # TODO: For now have no target transform
    target_xform_dict = {}
    for i, k in enumerate(include_classes):
        target_xform_dict[k] = i

    dataset = subsample_dataset(dataset, cls_idxs)

    dataset.target_transform = lambda x: target_xform_dict[x]

    return dataset

def subsample_instances(dataset, prop_indices_to_subsample=0.5):

    np.random.seed(0)
    subsample_indices = np.random.choice(range(len(dataset)), replace=False,
                                         size=(int(prop_indices_to_subsample * len(dataset)),))

    return subsample_indices

def subsample_dataset(dataset, idxs):

    mask = np.zeros(len(dataset)).astype('bool')
    mask[idxs] = True

    dataset.data = dataset.data[mask]
    dataset.uq_idxs = dataset.uq_idxs[mask]

    return dataset
whole_training_set = CustomCub2011(transform=None, train=True)

# 有标签训练集
# train_dataset_labelled = deepcopy(whole_training_set)
train_dataset_labelled = subsample_classes(deepcopy(whole_training_set), include_classes=range(100))
subsample_indices = subsample_instances(train_dataset_labelled)
train_dataset_labelled = subsample_dataset(train_dataset_labelled, subsample_indices)
print("len of labeled:", len(train_dataset_labelled))

# 无标签训练集
unlabelled_indices = set(whole_training_set.uq_idxs) - set(train_dataset_labelled.uq_idxs)
train_dataset_unlabelled = subsample_dataset(deepcopy(whole_training_set), np.array(list(unlabelled_indices)))
print("len of unlabeled:", len(train_dataset_unlabelled))

# 验证集
val_dataset = deepcopy(train_dataset_unlabelled)
val_dataset.transform = test_transform
print("len of val:", len(val_dataset))
test_loader_unlabelled = DataLoader(val_dataset, num_workers=8,batch_size=256, shuffle=False, pin_memory=False)

/opt/conda/envs/gcd/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


Files already downloaded and verified
len of labeled: 1500
len of unlabeled: 4494
len of val: 4494


In [27]:
import torch.nn.functional as F
class MyModel(torch.nn.Module):
    def __init__(self, class_num, feature_size, max_features_per_class):
        super(MyModel, self).__init__()
        self.class_num = class_num
        self.feature_size = feature_size
        self.max_features_per_class = max_features_per_class
        
        # 使用buffer来保存特征和类别计数
        self.register_buffer('feature_bank', torch.zeros(class_num, max_features_per_class, feature_size))
        self.register_buffer('feature_counts', torch.zeros(class_num, dtype=torch.long))

    def reset_feature_bank(self):
        """重置特征库在每个epoch开始时调用"""
        self.feature_bank.zero_()
        self.feature_counts.zero_()

    def update_feature_bank(self, proj, label, pseudo_label):
        probs = F.softmax(pseudo_label, dim=1)
        max_probs, predictions = probs.max(1)
        
        for feat, prob, pred in zip(proj, max_probs, predictions):
            if prob > 0.6:
                if self.feature_counts[pred] < self.max_features_per_class:
                    self.feature_bank[pred, self.feature_counts[pred]] = feat
                    self.feature_counts[pred] += 1
                    
    # def compute_loss(self, proj, pseudo_label, k):
    #     _, class_indices = torch.topk(pseudo_label, k, largest=False, dim=1)
    #     batch_size = proj.size(0)
    #     losses = torch.zeros(batch_size).to(proj.device)

    #     for i in range(batch_size):
    #         distances = []
    #         for class_idx in class_indices[i]:
    #             class_count = self.feature_counts[class_idx].item()
    #             if class_count > 0:
    #                 class_features = self.feature_bank[class_idx, :class_count]
    #                 distance_matrix = torch.sqrt((proj[i] - class_features).pow(2)).sum(dim=1)
    #                 distances.append(distance_matrix)

    #         if distances:
    #             total_distances = torch.cat(distances)
    #             loss_for_i = (1.0 / (total_distances + 1e-8)).sum()
    #             losses[i] = loss_for_i

    #     return losses.mean()
    def compute_loss(self, proj, pseudo_label, k):
        _, class_indices = torch.topk(pseudo_label, k, largest=False, dim=1)
        batch_size = proj.size(0)

        # Create a tensor to hold all distances
        all_distances = torch.zeros(batch_size, k).to(proj.device)

        for idx, class_set in enumerate(class_indices):
            class_counts = self.feature_counts[class_set]
            valid_mask = class_counts > 0
            valid_classes = class_set[valid_mask]
            
            # Initialize a tensor to hold distances for the current item in the batch across all valid classes
            distances_for_idx = torch.zeros(len(valid_classes)).to(proj.device)
            
            for j, valid_class in enumerate(valid_classes):
                # Extract features for the valid class based on class_counts
                valid_class_features = self.feature_bank[valid_class, :class_counts[valid_class].long()]
                
                # Expand dimensions for broadcasting
                expanded_proj = proj[idx].unsqueeze(0).expand_as(valid_class_features)

                distance = torch.sqrt((expanded_proj - valid_class_features).pow(2).sum(dim=-1))
                distances_for_idx[j] = distance.sum()

            all_distances[idx, valid_mask] = distances_for_idx

        # Calculate loss
        losses = (1.0 / (all_distances.sum(dim=-1) + 1e-8)).sum()
        
        return losses / batch_size

In [28]:
mymodel = MyModel(200, 256, 100).to('cuda')

In [16]:
model.eval()

proj, preds, targets = [], [], []
mask = np.array([])
for batch_idx, (images, label, _) in enumerate(test_loader_unlabelled):
    images = images.cuda(non_blocking=True)
    with torch.no_grad():
        imageproj, logits = model(images)
        preds.append(logits)
        targets.append(label)
        proj.append(imageproj)
        mask = np.append(mask, np.array([True if x.item() in range(100) else False for x in label]))

preds = torch.cat(preds,dim=0)
targets = torch.cat(targets,dim=0)
proj = torch.cat(proj,dim=0)

In [29]:
mymodel.update_feature_bank(proj, label, preds/0.1)
mymodel.feature_counts

tensor([ 1,  2,  8,  3,  3,  8,  4,  5,  2,  3,  3,  4,  5,  1,  4,  3,  4,  3,
         3,  3,  2,  2,  1,  7,  1,  3,  7,  2,  6,  2,  6,  6,  2,  4,  3,  1,
         2,  4,  3,  0,  3,  0,  1,  1,  2,  2,  2,  6,  0,  5,  5,  8,  2,  4,
         3,  2,  3,  2,  0,  2,  2,  0,  4,  2,  4,  3,  7,  0,  5,  3,  0,  4,
         4,  1,  5,  2,  8,  3,  2,  4,  7,  4,  7,  2,  5,  4,  3,  5,  6,  1,
         1,  5,  3,  1,  8,  1,  3,  2,  1,  3,  3,  7, 12, 14,  2, 13,  6, 10,
        11, 16, 10,  3,  7,  5,  8,  9, 11,  9,  8,  1, 16, 11, 15,  9,  7,  2,
         8,  6,  8,  9,  6,  6, 10,  5,  3,  7,  9, 11,  4,  3,  6,  8, 13,  4,
         4,  2,  6,  5, 10,  6, 10,  5,  4, 10,  4, 14,  5,  5,  4,  1, 16,  2,
         7,  7, 11, 10,  2,  2,  6,  8,  3,  9,  8, 11,  9, 18,  0,  9, 13,  6,
         6, 13,  8,  7, 12,  7,  3,  9,  8,  3,  6, 13,  0, 19,  9,  6, 11,  5,
         3,  7], device='cuda:0')

In [30]:
_, class_indices = torch.topk(preds, 10, largest=False, dim=1)
class_indices

tensor([[145,  58,  78,  ...,   9, 177, 196],
        [193,  32, 145,  ...,  92,  77,  65],
        [ 49,   3,   9,  ...,  53,   8,  96],
        ...,
        [ 55,  38,  96,  ...,  20,  90, 116],
        [ 91, 117, 110,  ...,  93,  56,   5],
        [ 85, 184,  11,  ..., 102,  94,  91]], device='cuda:0')

In [32]:
idx, class_set = 0, class_indices[0]
class_counts = mymodel.feature_counts[class_set]
class_counts

tensor([ 2,  0,  2,  6,  2,  4,  3,  3,  9, 11], device='cuda:0')

In [33]:
valid_mask = class_counts > 0
valid_classes = class_set[valid_mask]
valid_classes

tensor([145,  78,  28,  20,   6,  40,   9, 177, 196], device='cuda:0')

In [35]:
j, valid_class = 0, valid_classes[0]
valid_class

tensor(145, device='cuda:0')

In [37]:
valid_class_features = mymodel.feature_bank[valid_class, :mymodel.feature_counts[valid_class]]
valid_class_features.shape

torch.Size([2, 256])

In [38]:
expanded_proj = proj[idx].unsqueeze(0).expand_as(valid_class_features)
distance = torch.sqrt((expanded_proj - valid_class_features).pow(2).sum(dim=-1))

In [40]:
distance

tensor([1.3680, 1.3606], device='cuda:0')